# SRA metadata download
Adam Klie<br>11/23/2019<br>
Script to download relevant data for building, training and testing metadata prediction models

### Import needed packages

In [5]:
import xml.etree.ElementTree as ET
import pandas as pd

### 1. Download per sample metadata from [NCBI FTP](https://ftp.ncbi.nlm.nih.gov/sra/reports/Metadata/) <font color='red'>using previously parsed for now, will need to write code to parse</font>

In [ ]:
!wget -O ../data/NCBI_SRA_Metadata_20181202.tar.gz https://ftp.ncbi.nlm.nih.gov/sra/reports/Metadata/NCBI_SRA_Metadata_20181202.tar.gz

In [ ]:
#!tar -vxzf ../data/NCBI_SRA_Metadata_20181202.tar.gz
#sra_raw = pd.read_json('../data/NCBI_SRA_Metadata_20181202')

### 2. Sample and study technical metadata <font color='red'>need to find where this is hiding</font>

### 3. Download BioSample [attribute definitions](https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/)

In [3]:
!wget -O ../data/BioSampleAttributes.xml https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/?format=xml

--2019-10-17 18:23:24--  https://www.ncbi.nlm.nih.gov/biosample/docs/attributes/?format=xml
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘index.html?format=xml.1’

index.html?format=x     [  <=>               ] 942.58K  4.28MB/s    in 0.2s    

2019-10-17 18:23:25 (4.28 MB/s) - ‘index.html?format=xml.1’ saved [965201]



In [38]:
def parseXML(xmlfile):
  
    # create element tree object
    tree = ET.parse(xmlfile)
  
    # get root element
    root = tree.getroot()
    
    # create empty list for news items
    attribute_df = pd.DataFrame()
  
    # iterate news items
    for item in root.findall('./Attribute'):
        
        # empty news dictionary
        attribute = {}
  
        # iterate child elements of item
        for child in item:
            attribute[child.tag] = child.text
            
        # append news dictionary to news items list
        attribute_df = attribute_df.append(attribute, ignore_index = True)
      
    # return news items list
    return attribute_df


In [42]:
df = parseXML('BioSampleAttributes.xml').set_index('Name')

In [44]:
df.shape

(456, 5)

In [53]:
df.loc['disease']['Description']

'list of diseases diagnosed; can include multiple diagnoses. the value of the field depends on host; for humans the terms should be chosen from DO (Disease Ontology), free text for non-human. For DO terms, please see http://gemina.svn.sourceforge.net/viewvc/gemina/trunk/Gemina/ontologies/gemina_symptom.obo?view=log'

In [55]:
df.index

Index(['EDTA inhibitor tested', 'FAO classification', 'HIV status',
       'NARMS isolate number', 'Omics Observatory ID',
       'Population Description', 'Super Population Code',
       'Super Population Description', 'absolute air humidity',
       'affection status',
       ...
       'water content', 'water content of soil',
       'water content of soil method', 'water current',
       'water temperature regimen', 'watering regimen',
       'weight loss in last three months', 'wet mass', 'wind direction',
       'wind speed'],
      dtype='object', name='Name', length=456)

In [57]:
df.to_pickle('../BioSampleAttributes..pickle')